We have one row per chunk now. In this notebook we will convert each chunk's text into a numeric vector (embedding) so later can be used for: search by semantic similarity, get the retrieve relevant chunks and feed them to an LLM for RAG
 
It reads chunk-level documents from Unity Catalog, generates text embeddings using an external embedding model, and stores the resulting vectors as a Delta table for semantic search and retrieval-augmented generation (RAG).

Input and Output:
- Input table: databricks_rag_demo.default.azure_compute_doc_chunks
- Output table: databricks_rag_demo.default.azure_compute_doc_embeddings

Embedding strategy (important decisions)

For this project we will:
-   Use OpenAI-style embeddings (works with OpenAI or Azure OpenAI)
-   Generate embeddings in batches (not per row)
- Store embeddings as: ARRAY<FLOAT> (simple, portable)
- Keep metadata alongside vectors

This is the most common production pattern.

In [0]:
import os
import time
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType

In [0]:
chunks_df = spark.table(
    "databricks_rag_demo.default.azure_compute_doc_chunks"
)

In [0]:
%sql
SELECT doc_id, category, chunk_id, chunk_text FROM databricks_rag_demo.default.azure_compute_doc_chunks LIMIT 3

doc_id,category,chunk_id,chunk_text
virtual-machines/extensions/salt-minion.md,virtual-machines,8c083b147e62c8532ca84e73b344b0079e8b0cadebffb92a9edfd1546f525b8a,title salt minion for linux or windows azure vms description install salt minion on linux or windows vms using the vm extension ms topic concept article ms service azure virtual machines ms subservice extensions ms custom devx track arm template devx track azurecli devx track terraform linux related content ms author gabsta author gabstamsft ms date 08 18 2025 customer intent as a cloud administrator i want to install salt minion on my azure vms using vm extensions so that i can effectively manage and automate configurations across my infrastructure install salt minion on linux or windows vms using the vm extension prerequisites a microsoft azure account with one or more windows or linux vms a salt master either on premises or in a cloud that can accept connections from salt minions hosted on azure the salt minion vm extension requires that the target vm is connected to the internet in order to fetch salt packages include vm assist troubleshooting tools supported platforms azure vm running any of the following supported os ubuntu 20 04 22 04 x86_64 debian 10 11 x86_64 oracle linux 7 8 9 x86_64 rhel 7 8 9 x86_64 microsoft windows 10 11 pro x86_64 microsoft windows server 2012 r2 2016 2019 2022 datacenter x86_64 if you want another distro to be supported assuming salt supports it an issue can be filed on gitlab supported salt minion versions 3006 and up onedir extension details publisher name turtletraction oss linux extension name salt minion linux windows extension name salt minion windows salt minion settings master_address salt master address to connect to localhost by default minion_id minion id hostname by default salt_version salt minion version to install for example 3006 1 latest by default install salt minion using the azure portal 1 select one of your vms 2 in the left menu click extensions applications 3 click add 4 in the gallery type salt minion in the search bar 5 select the salt minion tile and click next 6 enter configuration parameters in the provided form see salt minion settings 7 click review create install salt minion using the azure cli to uninstall it install salt minion using the azure arm template install salt minion using terraform assuming that you have defined a vm resource in terraform named vm_ubuntu then use something like this to install the extension on it support for commercial support or assistance with salt
virtual-machines/extensions/salt-minion.md,virtual-machines,b31b8f47afa253eb5a55d07e79f3ae3aec9ba1cb892b4161ca98873280c7eff0,the azure cli to uninstall it install salt minion using the azure arm template install salt minion using terraform assuming that you have defined a vm resource in terraform named vm_ubuntu then use something like this to install the extension on it support for commercial support or assistance with salt you can visit the extension creator turtletraction the source code of this extension is available on gitlab for azure related issues you can file an azure support incident go to the azure support site and select get support
virtual-machines/extensions/backup-azure-sql-server-running-azure-vm.md,virtual-machines,e81e730867bab0e1924516a758e63f50decf7d4ec97fe49e4e0640fe734eb34a,title azure backup for sql server running in azure vm description in this article learn how to register azure backup in sql server running in an azure virtual machine ms topic concept article ms service azure virtual machines ms subservice extensions ms author gabsta ms reviewer jushiman author gabstamsft ms collection windows ms date 08 18 2025 customer intent as a database administrator i want to register azure backup for my sql server running in an azure vm so that i can ensure reliable backup and recovery of my database workloads azure backup for sql server running in azure vm azure backup amongst other offerings provides support for ba

In [0]:
# sanity check

from pyspark.sql import functions as F

# chunk_text is too long to show
chunks_df.select(
    "doc_id",
    "category",
    "chunk_index",
    F.substring("chunk_text", 1, 200).alias("chunk_preview")
).show(3, truncate=False)

+-----------------------------------------------------------------------+----------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|doc_id                                                                 |category        |chunk_index|chunk_preview                                                                                                                                                                                           |
+-----------------------------------------------------------------------+----------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|virtual-machines/extensions/salt-minion.md                             |virtual-machine

How to create Azure OpenAI service and get a api key:
- In Azure Portal: Create Azure OpenAI resource
- Deploy a model: text-embedding-3-small or text-embedding-ada-002
- Get:
	- Endpoint
	- API key


How to store API key in notebook (one-time)
- Workspace → Secrets
- Create scope: openai
  - Key: OPENAI_API_KEY
  - Value: your key

In [0]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.1 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.1/1.1 MB 63.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/113.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/73.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/78.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/361.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [0]:
## Initialize OpenAI emebdding client
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version="2024-02-01",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)

In [0]:
# We do NOT query embed for one row at a time, We do in batches for cost and performance.

def embed_texts(texts, model="text-embedding-3-small"):
    response = client.embeddings.create(
        model=model,
        input=texts
    )
    return [d.embedding for d in response.data]

In [0]:
# Collect chunks in manageable batches

BATCH_SIZE = 64

rows = chunks_df.select(
    "chunk_id",
    "doc_id",
    "category",
    "title",
    "url",
    "chunk_index",
    "chunk_text"
).collect()

print(f"Found {len(rows)} rows")

Found 5758 rows


In [0]:
#  Generate embeddings

embedded_rows = []

for i in range(0, len(rows), BATCH_SIZE):
    print(f"{i}")
    batch = rows[i:i + BATCH_SIZE]
    texts = [r.chunk_text for r in batch]

    embeddings = embed_texts(texts)

    for r, emb in zip(batch, embeddings):
        embedded_rows.append((
            r.chunk_id,
            r.doc_id,
            r.category,
            r.title,
            r.url,
            r.chunk_index,
            r.chunk_text,
            emb
        ))

        # first 10 batches
    if i > 10*BATCH_SIZE: break

    time.sleep(0.5)  # be polite to API

0
64
128
192
256
320
384
448
512
576
640
704


[Trace(request_id=tr-a7ed08f1d27942d79d688b840fc99724), Trace(request_id=tr-70e3746dea274d2e95f4f5ed6242f7a0), Trace(request_id=tr-f21c8bb53c1f458c88737fdd2b89f93b), Trace(request_id=tr-d76a90fbfcbf4076b48659a92c2fb3d2), Trace(request_id=tr-b9ca28528ec14c4facf6f4459e8e55d8), Trace(request_id=tr-df2459b6e0bf4be681ceca60f0e082b5), Trace(request_id=tr-b6bff65d7cab4c70bacb75c113a2468a), Trace(request_id=tr-0cfe7b59f735447faf204ce4fe52e9e5), Trace(request_id=tr-01168658d0c54ad099d7160a2c3d222a), Trace(request_id=tr-d4384a44be9543e9808ec32dae219f4f)]

In [0]:
# Create embeddings DataFrame

embeddings_df = spark.createDataFrame(
    embedded_rows,
    schema=[
        "chunk_id",
        "doc_id",
        "category",
        "title",
        "url",
        "chunk_index",
        "chunk_text",
        "embedding"
    ]
)

# Check vector length:
embeddings_df.select(F.size("embedding").alias("dim")).distinct().show()

+----+
| dim|
+----+
|1536|
+----+



In [0]:
(
    embeddings_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(
        "databricks_rag_demo.default.azure_compute_doc_embeddings"
    )
)

In [0]:
%sql
SELECT COUNT(*) FROM databricks_rag_demo.default.azure_compute_doc_embeddings;

count(1)
768


In [0]:
%sql
SELECT category, size(embedding) AS embedding_dim FROM databricks_rag_demo.default.azure_compute_doc_embeddings LIMIT 5;